# 05 - Backtesting Trading Strategies

This notebook demonstrates how to backtest trading strategies using historical data.

**Contents:**
1. Introduction to Backtesting
2. Data Preparation
3. Simple Moving Average Crossover Strategy
4. RSI Mean Reversion Strategy
5. MACD Strategy
6. Bollinger Band Strategy
7. Strategy Comparison
8. Parameter Optimization
9. Walk-Forward Analysis
10. Performance Metrics Analysis
11. Best Practices and Pitfalls

---

## ⚠️ DISCLAIMER

**Past performance is NOT indicative of future results. Backtesting results can be misleading due to:**
- Look-ahead bias
- Survivorship bias
- Overfitting to historical data
- Market regime changes
- Lack of transaction costs, slippage, and market impact

**Do NOT use these strategies for actual trading without:**
- Extensive out-of-sample testing
- Forward testing on live data
- Consultation with financial professionals
- Understanding of risk management

This notebook is for **educational purposes only**.

## 1. Introduction to Backtesting

**Backtesting** is the process of testing a trading strategy on historical data to evaluate its performance.

### Key Concepts:

- **Strategy**: A set of rules that determine when to buy and sell
- **Equity Curve**: Portfolio value over time
- **Drawdown**: Peak-to-trough decline in equity
- **Sharpe Ratio**: Risk-adjusted return metric
- **Win Rate**: Percentage of profitable trades
- **Profit Factor**: Gross profit / Gross loss

### Common Pitfalls:

1. **Overfitting**: Optimizing too much on historical data
2. **Look-ahead Bias**: Using future information in decisions
3. **Survivorship Bias**: Testing only on stocks that survived
4. **Ignoring Costs**: Not accounting for commissions and slippage

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from src.data.fetcher import get_stock_data
from src.backtesting import (
    BacktestEngine,
    quick_backtest,
    SMACrossover,
    RSIMeanReversion,
    MACDStrategy,
    BollingerBandStrategy,
    TrendFollowing,
    MultiStrategyCombo,
    PerformanceMetrics,
    plot_equity_curve,
    plot_drawdown,
    plot_monthly_returns
)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("Libraries imported successfully!")

## 2. Data Preparation

Fetch historical data for backtesting.

In [ ]:
# Fetch data
ticker = 'AAPL'
start_date = '2020-01-01'
end_date = '2024-01-01'

print(f"Fetching data for {ticker} from {start_date} to {end_date}...")
df = get_stock_data(ticker, start=start_date, end=end_date)

# Prepare data for backtesting (OHLCV only)
backtest_data = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

print(f"\nData shape: {backtest_data.shape}")
print(f"Date range: {backtest_data.index[0].date()} to {backtest_data.index[-1].date()}")
print(f"\nFirst few rows:")
backtest_data.head()

In [ ]:
# Visualize price data
plt.figure(figsize=(14, 6))
plt.plot(backtest_data.index, backtest_data['Close'], linewidth=2)
plt.title(f'{ticker} Price History', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Simple Moving Average (SMA) Crossover Strategy

**Strategy Rules:**
- **Buy Signal**: When fast SMA crosses above slow SMA (golden cross)
- **Sell Signal**: When fast SMA crosses below slow SMA (death cross)

**Parameters:**
- Fast SMA: 50 days
- Slow SMA: 200 days

In [ ]:
# Run SMA Crossover backtest
print("=" * 70)
print("SMA CROSSOVER STRATEGY")
print("=" * 70)

sma_stats = quick_backtest(
    backtest_data,
    SMACrossover,
    initial_cash=10000,
    commission=0.002,  # 0.2% commission
    strategy_params={'fast_period': 50, 'slow_period': 200},
    plot=True
)

## 4. RSI Mean Reversion Strategy

**Strategy Rules:**
- **Buy Signal**: When RSI < 30 (oversold)
- **Sell Signal**: When RSI > 70 (overbought)

**Parameters:**
- RSI Period: 14 days
- Lower threshold: 30
- Upper threshold: 70

In [ ]:
# Run RSI Mean Reversion backtest
print("=" * 70)
print("RSI MEAN REVERSION STRATEGY")
print("=" * 70)

rsi_stats = quick_backtest(
    backtest_data,
    RSIMeanReversion,
    initial_cash=10000,
    commission=0.002,
    strategy_params={'rsi_period': 14, 'rsi_lower': 30, 'rsi_upper': 70},
    plot=True
)

## 5. MACD Strategy

**Strategy Rules:**
- **Buy Signal**: When MACD line crosses above signal line
- **Sell Signal**: When MACD line crosses below signal line

**Parameters:**
- Fast EMA: 12 days
- Slow EMA: 26 days
- Signal line: 9 days

In [ ]:
# Run MACD Strategy backtest
print("=" * 70)
print("MACD STRATEGY")
print("=" * 70)

macd_stats = quick_backtest(
    backtest_data,
    MACDStrategy,
    initial_cash=10000,
    commission=0.002,
    plot=True
)

## 6. Bollinger Band Strategy

**Strategy Rules:**
- **Buy Signal**: When price touches lower band (oversold)
- **Sell Signal**: When price touches upper band (overbought) or reaches middle band

**Parameters:**
- Period: 20 days
- Standard deviations: 2

In [ ]:
# Run Bollinger Band Strategy backtest
print("=" * 70)
print("BOLLINGER BAND STRATEGY")
print("=" * 70)

bb_stats = quick_backtest(
    backtest_data,
    BollingerBandStrategy,
    initial_cash=10000,
    commission=0.002,
    plot=True
)

## 7. Strategy Comparison

Compare all strategies side-by-side.

In [ ]:
# Compare all strategies
engine = BacktestEngine(backtest_data, initial_cash=10000, commission=0.002)

comparison = engine.compare_strategies({
    'SMA Crossover': SMACrossover,
    'RSI Mean Reversion': RSIMeanReversion,
    'MACD': MACDStrategy,
    'Bollinger Bands': BollingerBandStrategy,
    'Trend Following': TrendFollowing,
    'Multi-Strategy': MultiStrategyCombo
})

print("\n" + "=" * 70)
print("STRATEGY COMPARISON")
print("=" * 70)
print(comparison)

# Highlight best strategy
best_strategy = comparison['Return [%]'].idxmax()
best_return = comparison.loc[best_strategy, 'Return [%]']
print(f"\n🏆 Best Strategy: {best_strategy} with {best_return:.2f}% return")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Returns
comparison['Return [%]'].plot(kind='barh', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title('Total Return', fontweight='bold')
axes[0, 0].set_xlabel('Return (%)')
axes[0, 0].grid(True, alpha=0.3)

# Sharpe Ratio
comparison['Sharpe Ratio'].plot(kind='barh', ax=axes[0, 1], color='green')
axes[0, 1].set_title('Sharpe Ratio', fontweight='bold')
axes[0, 1].set_xlabel('Sharpe Ratio')
axes[0, 1].grid(True, alpha=0.3)

# Max Drawdown
comparison['Max Drawdown [%]'].plot(kind='barh', ax=axes[1, 0], color='red')
axes[1, 0].set_title('Maximum Drawdown', fontweight='bold')
axes[1, 0].set_xlabel('Drawdown (%)')
axes[1, 0].grid(True, alpha=0.3)

# Win Rate
comparison['Win Rate [%]'].plot(kind='barh', ax=axes[1, 1], color='orange')
axes[1, 1].set_title('Win Rate', fontweight='bold')
axes[1, 1].set_xlabel('Win Rate (%)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Parameter Optimization

Optimize strategy parameters using grid search.

**⚠️ Warning**: Parameter optimization can lead to overfitting!

In [ ]:
# Optimize SMA Crossover parameters
print("Optimizing SMA Crossover parameters...")
print("This may take a few minutes...\n")

engine_opt = BacktestEngine(backtest_data, initial_cash=10000, commission=0.002)

optimized_stats = engine_opt.run_backtest(
    SMACrossover,
    optimize=True,
    optimize_params={
        'fast_period': range(20, 80, 10),
        'slow_period': range(100, 250, 25)
    },
    maximize='Sharpe Ratio',
    return_heatmap=True
)

print("\n" + "=" * 70)
print("OPTIMIZED PARAMETERS")
print("=" * 70)
print(f"Fast Period: {optimized_stats._strategy.fast_period}")
print(f"Slow Period: {optimized_stats._strategy.slow_period}")
print(f"\nOptimized Sharpe Ratio: {optimized_stats['Sharpe Ratio']:.2f}")
print(f"Optimized Return: {optimized_stats['Return [%]']:.2f}%")

## 9. Walk-Forward Analysis

Walk-forward analysis helps prevent overfitting by:
1. Optimizing on a training window
2. Testing on the next out-of-sample window
3. Moving forward and repeating

This simulates real-world trading where you periodically re-optimize.

In [ ]:
# Perform walk-forward analysis
print("Performing walk-forward analysis...")
print("This will take several minutes...\n")

wf_results = engine.walk_forward_analysis(
    SMACrossover,
    train_period=252,  # 1 year training
    test_period=63,    # 3 months testing
    optimize_params={
        'fast_period': range(30, 70, 10),
        'slow_period': range(150, 250, 25)
    },
    maximize='Sharpe Ratio'
)

print("\n" + "=" * 70)
print("WALK-FORWARD ANALYSIS RESULTS")
print("=" * 70)
print(wf_results)

# Summary statistics
print(f"\n📊 Summary:")
print(f"Average Return: {wf_results['Return [%]'].mean():.2f}%")
print(f"Average Sharpe: {wf_results['Sharpe Ratio'].mean():.2f}")
print(f"Win Rate: {(wf_results['Return [%]'] > 0).sum() / len(wf_results) * 100:.1f}%")

In [ ]:
# Plot walk-forward results
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Returns over time
axes[0].bar(range(len(wf_results)), wf_results['Return [%]'], 
            color=['green' if x > 0 else 'red' for x in wf_results['Return [%]']])
axes[0].set_title('Walk-Forward Returns by Period', fontweight='bold')
axes[0].set_ylabel('Return (%)')
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[0].grid(True, alpha=0.3)

# Sharpe Ratio over time
axes[1].plot(range(len(wf_results)), wf_results['Sharpe Ratio'], 
             marker='o', linewidth=2, color='blue')
axes[1].set_title('Walk-Forward Sharpe Ratio by Period', fontweight='bold')
axes[1].set_ylabel('Sharpe Ratio')
axes[1].set_xlabel('Period')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Performance Metrics Analysis

Deep dive into performance metrics.

In [ ]:
# Run a strategy and get detailed metrics
engine_metrics = BacktestEngine(backtest_data, initial_cash=10000, commission=0.002)
stats = engine_metrics.run_backtest(SMACrossover)

# Get trades
trades = engine_metrics.get_trades()

print("=" * 70)
print("DETAILED TRADE LOG")
print("=" * 70)
print(trades.head(10))

# Calculate additional metrics
equity_curve = stats['_equity_curve']['Equity']
returns = equity_curve.pct_change().dropna()

comprehensive_metrics = PerformanceMetrics.calculate_comprehensive_metrics(
    equity_curve,
    trades,
    initial_cash=10000
)

print("\n" + "=" * 70)
print("COMPREHENSIVE METRICS")
print("=" * 70)
print(comprehensive_metrics)

In [ ]:
# Plot equity curve
plot_equity_curve(equity_curve, title=f'{ticker} - SMA Crossover Equity Curve')

In [ ]:
# Plot drawdown
plot_drawdown(equity_curve, title=f'{ticker} - SMA Crossover Drawdown')

In [ ]:
# Plot monthly returns
plot_monthly_returns(returns, title=f'{ticker} - SMA Crossover Monthly Returns')

## 11. Best Practices and Pitfalls

### ✅ Best Practices:

1. **Use Out-of-Sample Testing**: Always test on unseen data
2. **Account for Costs**: Include realistic commissions and slippage
3. **Walk-Forward Analysis**: Periodically re-optimize and test
4. **Risk Management**: Use stop losses and position sizing
5. **Multiple Metrics**: Don't optimize for just one metric
6. **Market Regimes**: Test strategies in different market conditions
7. **Paper Trading**: Test strategies with simulated real-time data before live

### ❌ Common Pitfalls:

1. **Overfitting**: Excessive parameter optimization
2. **Look-Ahead Bias**: Using future data in calculations
3. **Survivorship Bias**: Testing only on currently traded stocks
4. **Ignoring Costs**: Not accounting for slippage and commissions
5. **Data Mining**: Testing hundreds of strategies and cherry-picking
6. **Regime Changes**: Assuming past patterns will continue
7. **Psychological Factors**: Ignoring emotional challenges of real trading

### 📝 Checklist Before Live Trading:

- [ ] Strategy tested on out-of-sample data
- [ ] Walk-forward analysis performed
- [ ] Realistic transaction costs included
- [ ] Risk management rules defined
- [ ] Position sizing rules established
- [ ] Maximum drawdown acceptable
- [ ] Paper trading completed successfully
- [ ] Psychological preparedness assessed
- [ ] Capital you can afford to lose

---

## Summary

In this notebook, we:

1. ✅ Learned backtesting fundamentals
2. ✅ Implemented multiple trading strategies
3. ✅ Compared strategy performance
4. ✅ Optimized strategy parameters
5. ✅ Performed walk-forward analysis
6. ✅ Analyzed comprehensive performance metrics
7. ✅ Learned best practices and common pitfalls

**Remember**: Backtesting is a tool for learning and research, not a guarantee of future profits!

---

### Next Steps:

- Explore the **06_dashboard.ipynb** for interactive analysis
- Create your own custom strategies
- Test on different stocks and time periods
- Implement risk management rules
- Paper trade before risking real capital